In [6]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from Supply import Supply

In [2]:
model = Supply()
folder = "/Users/emmaknippel/Desktop/POLIT 10 SEM/Dynamic Programming/Replicating the Innovators Dilemma/1 Summary Statistics/Data"
model.setup(folder)

In [3]:
results = model.MC()

dPdQ = results['dPdQ']
ePQ = results['ePQ']
ABCDEFGH = results['ABCDEFGH']
q = results['q']
MC = results['MC']

In [4]:
print(q[:, 1])

q_copy = q.copy()
q_copy[:, 1] = q[:, 0]
q_copy[0, 3] = q[1, 3]
q_copy[:, 2] = q_copy[:, 3]


[   0.            0.          102.10909091  224.37272727  279.19090909
  486.58        728.5         811.96        844.91111111  873.58571429
  407.2         258.15        159.18         76.34         83.06
  913.         1114.56       1017.5       ]


In [ ]:
par = model.par
print(par.State[13,0])
print(par.State[13,1])
print(par.State[13,2])

In [6]:
print(MC[13,0])
print(MC[13,1])
print(MC[13,2])
print(MC[13,3])

1.1213105727517398
-2.916521500397822
0.34984557037628317
0.3995721848620778


In [12]:
alpha1 = -3.28454
alpha2 = 0.909773
alpha3 = 1.204684
Mkt = 56638.8
X_o = 6.490788
X_n = 6.240812
Xe_o = -0.4157396
Xe_n = 0.4135973
YearDummy = -5.653561
MC_o = 1.1213105727517398
MC_n = -2.916521500397822
No = 1
Nb = 4
Nn = 5

def foc(q):
    Qo = No * q[13,0] + Nb * q[13,1]
    Qn = Nb * q[13,2] + Nn * q[13,3]
    Q0 = Mkt - Qo - Qn

    Po = (-1 / alpha1) * (-np.log(Qo / Q0) + alpha2 * 0 + alpha3 * X_o + Xe_o + YearDummy)
    Pn = (-1 / alpha1) * (-np.log(Qn / Q0) + alpha2 * 1 + alpha3 * X_n + Xe_n + YearDummy)

    dPoQo = (Qo + Q0) / (alpha1 * Qo * Q0)
    dPnQo = 1 / (alpha1 * Q0)
    dPoQn = 1 / (alpha1 * Q0)
    dPnQn = (Qn + Q0) / (alpha1 * Qn * Q0)

    foc_o  = Po + dPoQo * q[13,0] - MC_o
    foc_bo = Po + dPoQo * q[13,1] + dPnQo * q[13,2] - MC_o
    foc_bn = Pn + dPnQn * q[13,2] + dPoQn * q[13,1] - MC_n
    foc_n  = Pn + dPnQn * q[13,3] - MC_n  

    F = foc_o**2 + foc_bo**2 + foc_bn**2 + foc_n**2

    return F

foc(q)

27.812590370735805

In [15]:
par = model.par
State = par.State
T = par.T

for t in range(T-2, -1, -1):
    print(f'Period t = {t}, No = {State[t,0]}')
    ststeprime = State[t+1,0]

Period t = 16, No = 1
Period t = 15, No = 1
Period t = 14, No = 1
Period t = 13, No = 1
Period t = 12, No = 1
Period t = 11, No = 2
Period t = 10, No = 2
Period t = 9, No = 2
Period t = 8, No = 3
Period t = 7, No = 3
Period t = 6, No = 3
Period t = 5, No = 4
Period t = 4, No = 4
Period t = 3, No = 5
Period t = 2, No = 10
Period t = 1, No = 11
Period t = 0, No = 11


In [4]:
x = 0
x += 4.35
print(type(x))

<class 'float'>


In [12]:
from math import factorial

def fun1(z6, z7, z8, z9, z10, No, Nb, Nn, Npe, Npe_prime, Vprime):
    '''
    Input variables:
        z6, z7, z8, z9, z10 = choice probabilities, fun6 - fun 10
        No = integer,  # Old-only firms
        Nb = integer, # of Both firms
        Nn = integer, # of New firms
        Npe = integer, # of Potential Entrants
        Npe_prime = integer, # of Potential Entrants next period
        Vprime = array of size 6480,1 with value function results over time, type and # of firms

    Output:
        z1 = EV of staying for Old-only firms
    '''

    prhs = [z6, z7, z8, z9, z10, No, Nb, Nn, Npe, Npe_prime, Vprime]
    nrhs = len(prhs)

    # checks for correct inputs
    if nrhs != 11:
        raise Warning(f'Error fun1: 11 input arguments required, only {nrhs} given')
    if prhs[10].size != 6480:
        raise Warning(f'Error fun1: Vprime must have 6480 rows, it has {prhs[10].size}')
    if np.isnan(prhs[10]).any():
        raise Warning(f'Error fun1: Vprime must have 6480 elements')


    def getBA1(z6, z7, z8, z9, z10, No, Nb, Nn, Npe):
        '''
        Objective: compute BA1 (??)

            xo = # of exits, old firms
            eb = # entry of both - aka # of adopts
            xb = # exit both firms
            xn = # exit new firms
            en = # entry potential firms
        '''

        BA1 = np.zeros((12*12*12*15*5))  # xo, eb, xb, xn, en
        for xo in range(No):
            for eb in range(No-xo):
                for xb in range(Nb+1):
                    for xn in range(Nn+1):
                        for en in range(Npe+1):
                            if No > 1:      # if number of old firms > 1
                                            # Intuitively: Ba1[xo][eb][xb][xn][en]
                                BA1[xo + 12*eb +(12*12)*xb + (12*12*12)*xn + (12*12*12*15)*en] = (factorial(No - 1) / (factorial(xo) * factorial(No - 1 - xo))) \
                                * (factorial(No - 1 - xo) / (factorial(eb) * factorial(No - 1 - xo - eb))) \
                                * z6**xo * z7**eb * ((1 - z6 - z7)**(No - 1 - xo - eb)) \
                                * (factorial(Nb) / (factorial(xb) * factorial(Nb - xb))) \
                                * z8**xb * ((1 - z8)**(Nb - xb)) \
                                * (factorial(Nn) / (factorial(xn) * factorial(Nn - xn))) \
                                * z9**xn * ((1 - z9)**(Nn - xn)) \
                                * (factorial(Npe) / (factorial(en) * factorial(Npe - en))) \
                                * z10**en * ((1 - z10)**(Npe - en))
                            else:
                                BA1[0 + 12*0 + (12*12)*xb + (12*12*12)*xn + (12*12*12*15)*en] = \
                                (factorial(Nb) / (factorial(xb) * factorial(Nb - xb))) \
                                * z8**xb * ((1 - z8)**(Nb - xb)) \
                                * (factorial(Nn) / (factorial(xn) * factorial(Nn - xn))) \
                                * z9**xn * ((1 - z9)**(Nn - xn)) \
                                * (factorial(Npe) / (factorial(en) * factorial(Npe - en))) \
                                * z10**en * ((1 - z10)**(Npe - en))
        return BA1


    def getBS1(No, Nb, Nn, Npe, Npe_prime):
    # Step 2: map BA1 to future state probabilities BS1
        npe_prime = Npe_prime

        BA1 = getBA1(z6,z7,z8,z9,z10,No,Nb,Nn,Npe)

        BS1 = np.zeros((12*12*15*5))  # no', nb', nn', npe'
        for xo in range(No):
            for eb in range(No - xo):
                for xb in range(Nb):
                    for xn in range(Nn + 1):
                        for en in range(Npe + 1):
                            xo = max(xo, 0)
                            eb = max(eb, 0)

                            no_prime = No - xo - eb
                            no_prime = max(0, no_prime)
                            no_prime = min(no_prime, 11)

                            nb_prime = Nb - xb + eb
                            nb_prime = max(0, nb_prime)
                            nb_prime = min(nb_prime, 11)

                            nn_prime = Nn - xn + en
                            nn_prime = max(0, nn_prime)
                            nn_prime = min(nn_prime, 14)

                            BS1[no_prime + 12*nb_prime + (12*12)*nn_prime + (12*12*15)*npe_prime] += \
                                BA1[xo + 12*eb + (12*12)*xb + (12*12*12)*xn + (12*12*12*15)*en]
        return BS1

    def getEV1(No, Nb, Nn, Npe, Npe_prime, Vprime):
        npe_prime = Npe_prime
        BS1 = getBS1(No, Nb, Nn, Npe, Npe_prime)

        EV1 = 0.0  # Solution container
        for no_prime in range(0,12):
            for nb_prime in range(0,12):
                for nn_prime in range(0,15):
                    EV1 += BS1[no_prime + 12*nb_prime + (12*12)*nn_prime + (12*12*15)*npe_prime] \
                        * Vprime[0 + 3*no_prime + (3*12)*nb_prime + (3*12*12)*nn_prime]

        return EV1

    # Initialize result container
    # z1 = np.zeros(1)
    # BA1 = getBA1(z6, z7, z8, z9, z10, No, Nb, Nn, Npe)
    # BS1 = getBS1(No, Nb, Nn, Npe, Npe_prime, BA1)
    z1 = getEV1(No, Nb, Nn, Npe, Npe_prime, Vprime)
    print(z1)

    return z1

# Example usage
z6 = 0.1
z7 = 0.2
z8 = 0.3
z9 = 0.4
z10 = 0.5
No = 5
Nb = 3
Nn = 2
Npe = 4
Npe_prime = 3
Vprime = np.random.uniform(low=0.00001, high=20000, size=6480)  # Example value function results

result = fun1(z6, z7, z8, z9, z10, No, Nb, Nn, Npe, Npe_prime, Vprime)

9886.46425524372


In [16]:
LL = np.full((18-1,4), 3)

#np.sum(LL)
np.sum(np.sum(LL))

204